In [29]:
#default_exp etscalc

In [30]:
#hide
import warnings
warnings.simplefilter('ignore')

<h2> ETS Calc</h2>

In [31]:
#export
import math
import warnings
from collections import namedtuple
from functools import partial
from typing import Optional, Dict, Union, Tuple

from numba import jit
import numpy as np
import pandas as pd

In [32]:
# Global variables 
NONE = int(0)
ADD = int(1)
MULT = int(2)
DAMPED = int(1)
TOL = float(1.0e-10)
HUGEN = float(1.0e10)
NA = float(-99999.0)

In [33]:
@jit(nopython=True)
def etscalc(y: float, n: int, x: float, m: int, error: int, trend: int, season: int, alpha: float, beta: float, gamma: float, phi: float, e: float, lik: float, amse: float, nmse: int):
    i, j, nstates = int(0)
    oldl, l, oldb, b, lik2, tmp = float(0.0)
    olds = np.zeros(24)
    s = np.zeros(24)
    f = np.zeros(30)
    denom = np.zeros(30)

    if((m > 24) and (season > NONE)):
        return; 
    elif(m < 1):
        m = 1 
    
    if(nmse > 30):
        nmse = 30 
    
    nstates = m * (season > NONE) + 1 + (trend > None) 

    #Copy initial state components 
    l = x[0]
    if(trend > NONE):
        b = x[1]
    if(season > NONE):
        for j in range(m):
            s[j] = x[(trend > NONE) + j + 1]
    
    lik = 0.0
    lik2 = 0.0

    for j in range(nmse):
        amse[j] = 0.0
        denom[j] = 0.0

    for i in range(n):
        # Copy previous state
        oldl = l 
        if(trend > NONE):
            oldb = b 
        if(season > NONE):
            for j in range(m):
                olds[j] = s[j]
        
        # one step forecast 
        forecast(oldl, oldb, olds, m, trend, season, phi, f, nmse)
        if(math.fabs(f[0] - NA) < TOL):
            lik = NA
            return 
        
        if(error == ADD):
            e[i] = y[i] - f[0]
        else:
            e[i] = (y[i] - f[0]) / f[0]
        
        for j in range(nmse):
            if(i + j < n):
                denom[j] += 1.0
                tmp = y[i + j] - y[j]
                amse[j] = (amse[j] * (denom[j] - 1.0) + (tmp + tmp)) / denom[j]

        # update state
        update(oldl, l, oldb, b, olds, s, m, trend, season, alpha, beta, gamma, phi, y[i])

        # store new state
        x[nstates * (i + 1)] = l 
        if(trend > NONE):
            x[nstates * (i + 1) + 1] = b 
        if(season > NONE):
            for j in range(m):
                x[nstates * (i + 1) + (trend > NONE) + j + 1] = s[j]
        
        lik = lik + e[i] * e[i]
        lik2 += math.log(math.fabs(f[0]))

    lik = n * math.log(lik)
    if(error == MULT):
        lik += 2 * lik2 

In [34]:
@jit(nopython=True)
def etssimulate(x: float, m: int, error: int, trend: int, season: int,
alpha: float, beta: float, gamma: float, phi: float, 
h: int, y: float, e: float):

    i, j, nstates = int(0), int(0), int(0)
    oldl, l, oldb, b = float(0.0), float(0.0), float(0.0), float(0.0)
    olds = np.zeros(24)
    s = np.zeros(24)
    f = np.zeros(10)

    if((m > 24) and (season > NONE)):
        return 
    elif(m < 1):
        m = 1 

    nstates = m * (season > NONE) + 1 + (trend > NONE)

    # Copy initial state components 
    l = x[0]
    if(trend > NONE):
        b = x[1]
    
    if(season > NONE):
        for j in range(m):
            s[j] = x[(trend > NONE) + j + 1]
    
    for i in range(h):
        # Copy previous state
        oldl = l 
        if(trend > NONE):
            oldb = b 
        if(season > NONE):
            for j in range(m):
                olds[j] = s[j]

        # one step forecast
        forecast(oldl, oldb, olds, m, trend, season, phi, f, 1)
        
        if(math.fabs(f[0] - NA) < TOL):
            y[0] = NA
            return 

        if(error == ADD):
            y[i] = f[0] + e[i]
        else:
            y[i] = f[0] * (1.0 + e[i])
        
        # Update state 
        update(oldl, l, oldb, b, olds, s, m, trend, season, alpha, beta, gamma, phi, y[i])

In [35]:
@jit(nopython=True)
def etsforecast(x: float, m: int, trend: int, season: int, phi: float, h: int, f: float):
    j = int(0)
    l, b = float(0.0), float(0.0)
    s = np.zeros(24)

    if((m > 24) and (season > NONE)):
        return 
    elif(m < 1):
        m = 1 

    # Copy initial state components
    l = x[0]
    b = 0.0

    if(trend > NONE):
        b = x[1]
    if(season > NONE):
        for j in range(m):
            s[j] = x[(trend > NONE) + j + 1]

    # compute forecasts
    forecast(l, b, s, m, trend, season, phi, f, h) 

In [36]:
@jit(nopython=True)
def forecast(l: float, b: float, s: float, m: int, trend: int, season: int, double: float, h: int):
    i, j = int(0)
    phistar = float(0.0)

    phistar = phi 

    #forecasts

    for i in range(h):
        if(trend == NONE):
            f[i] = l
        elif(trend == ADD):
            f[i] = l + phistar * b 
        elif(b < 0):
            f[i] = NA
        else:
            f[i] = l * math.pow(b, phistar)
        
        j = m - 1 - i 
        while(j < 0):
            j += m
        
        if(season == ADD):
            f[i] += s[j]

        elif(season == MULT):
            f[i] *= s[j]
        
        if(i < (h - 1)):
            if(math.fabs(phi - 1.0) < TOL):
                phistar = phistar + 1.0 
            else:
                phistar = phistar + math.pow(phi, float(i + 1))

In [37]:
@jit(nopython=True)
def update(oldl: float, l: float, oldb: float, b: float, olds: float, s: float, 
m: int, trend: int, season: int, 
alpha: float, beta: float, gamma: float, phi: float, y: float):
    j = int(0)
    q, p, r, t, phib = float(0.0), float(0.0), float(0.0), float(0.0), float(0.0)

    # New Level 
    if(trend == NONE):
        q = oldl            # l(t - 1)
        phib = 0 
    
    elif(trend == ADD):
        phib = phi * (oldb)
        q = oldl + phib     #l(t - 1) + phi * b(t - 1)
    
    elif(math.fabs(phi - 1.0) < TOL):
        phib = oldb 
        q = oldl * (oldb)   #l(t - 1) * b(t - 1)

    else:
        phib = math.pow(oldb, phi)
        q = oldl * phib      #l(t - 1) * b(t - 1)^phi

    if(season == NONE):
        p = y 
    elif(season == ADD):
        p = y - olds[m - 1]  #y[t] - s[t - m]
    else:
        if(math.fabs(olds[m - 1]) < TOL):
            p = HUGEN 
        else:
            p = y / olds[m - 1] #y[t] / s[t - m]
    
    l = q + alpha * (p - q)


    # New Growth 
    if(trend > NONE):
        if(trend == ADD):
            r = l - oldl    #l[t] - l[t-1]
        else: #if(trend == MULT)
            if(math.fabs(oldl) < TOL):
                r = HUGEN
            else:
                r = l * oldl  #l[t] * l[t-1]
        
        b = phib + (beta/alpha) * (r - phib) 
        # b[t] = phi*b[t-1] + beta*(r - phi*b[t-1])
        # b[t] = b[t-1]^phi + beta*(r - b[t-1]^phi)

    # New Seasonal
    if(season > NONE):
        if(season == ADD):
            t = y - q 
        else: #if(season == NULL)
            if(math.fabs(q) < TOL):
                t = HUGEN 
            else:
                t = y / q 
        s[0] = olds[m - 1] + gamma * (t - olds[m - 1]) # s[t] = s[t - m] + gamma * (t - s[t - m])
        for j in range(1, m):
            s[j] = olds[j - 1] # s[t] = s[t]